In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# Legacy Method ( Using Agent Executor )

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)

In [6]:
search.invoke("what is time in yogyakarta Now")

[{'url': 'https://www.thetimenow.com/indonesia/yogyakarta',
  'content': 'Current local time and geoinfo in Yogyakarta, Indonesia . The Time Now is a reliable tool when traveling, calling or researching. The Time Now provides accurate (US network of cesium clocks) synchronized time and accurate time services in Yogyakarta, Indonesia.'},
 {'url': 'https://time.is/Yogyakarta',
  'content': 'The time in Yogyakarta is 12 hours ahead of the time in New York when New York is on standard time, and 11 hours ahead of the time in New York when New York is on daylight saving time. Yogyakarta does not change between summer time and winter time. The IANA time zone identifier for Yogyakarta is Asia/Jakarta.'}]

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = Chroma.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
from langchain.tools.retriever import create_retriever_tool

In [10]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [11]:
tools = [search, retriever_tool]

In [13]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0)


In [14]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)


In [17]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [18]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can assist with testing in several ways:\n\n1. **Monitoring and Evaluation**: LangSmith allows you to closely monitor and evaluate your applications, which helps ensure that they function correctly and meet performance standards.\n\n2. **Logging Traces**: You can log traces of your application to LangSmith, which helps in tracking the behavior of your application during testing. This is particularly useful for identifying issues and understanding how your application processes inputs.\n\n3. **Evaluation Framework**: LangSmith provides an evaluation framework that allows you to run tests against your application. You can define evaluators to check if the outputs of your application match expected results.\n\n4. **Integration with LangChain**: While LangSmith can work independently, it also integrates well with LangChain, allowing users to leverage its capabilities for applications built on that framework.\n\nOverall

In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [20]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [22]:
agent_with_chat_history.invoke(
    {"input": "who am i ?"},
    config={"configurable": {"session_id": "user_123"}},
)

{'input': 'who am i ?',
 'chat_history': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': "You mentioned that your name is Bob. If there's something specific you'd like to know about yourself or if you have any questions, feel free to ask!"}

In [ ]:
# Using sqlite to persist the chat history using sqlite db but you can choose another db like redis, mongodb etc

In [23]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_memory(session_id: str):
    return SQLChatMessageHistory( session_id="user_123", connection_string="sqlite:///sqlite.db")

In [35]:
agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [36]:
agent_with_history.invoke(
    {"input": "what's my last question ?"},
    config={"configurable": {"session_id": "user_123"}},
)

{'input': "what's my last question ?",
 'chat_history': [HumanMessage(content='who am i ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. Therefore, I can't determine who you are. If you provide me with some information about yourself or what you're looking for, I would be happy to help!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='i am bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Who am i ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. Therefore, I can't determine who you are. If you provide me with some information about yourself or what y

## Create openai function

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

## Create React Agent

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Agent Langgraph